### Helper Utils

In [104]:
def suffixArrayConstruction(text):
    ordered_pairs = sorted([(text[i:],i) for i in range(len(text))])
    suffix_array = [pair[1] for pair in ordered_pairs]
    return suffix_array
def greaterThan(p, p_):
    if (p_[:len(p)] < p):
        return True

    return False

def equalPatterns(p,p_):
    if p_[:len(p)] == p :
        return True
    return False

### Pattern Matching with the Suffix Array

In [120]:
def patternMatchingSA(text, patterns,suffixArr):

    minidx = 0
    maxidx = len(text)-1
    while minidx <= maxidx:
        mididx = (minidx+maxidx)//2
        if greaterThan(pattern,text[suffixArr[mididx]:]):
            minidx = mididx+1
        else:
            maxidx = mididx-1

    if equalPatterns(pattern,text[suffixArr[mididx]:suffixArr[mididx]+len(pattern)]):
        first = suffixArr[minidx]
    else:
        return 

    maxidx = len(text)-1
    while minidx <= maxidx:
        mididx = (minidx+maxidx)//2
        if text[suffixArr[mididx]:].startswith(pattern):
            minidx = mididx+1
        else:
            maxidx = mididx-1
    last= suffixArr[maxidx]
    return (first, last)


In [119]:
f = open('test.txt')
s = f.readline().strip()
patterns = [i.strip() for i in f.readlines()]
sArray = suffixArrayConstruction(s)
positions = []
for pattern in patterns:
    pair = patternMatchingSA(s, pattern, sArray)
    if pair is not None:
        positions.append(pair)
print(' '.join([str(pos) for pos in sorted([x for t in positions for x in t])]))


1 4 11 15


### Construct the Burrows-Wheeler Transform of a String

In [2]:
def constructBWTstring(text):
    cycles = []

    for i in range(len(text)):
        shift = text[i:] + text[:i]
        cycles.append(shift)

    cycles = sorted(cycles)
    bwt = ''
    for cycle in cycles:
        bwt += cycle[-1]
    print(bwt)

constructBWTstring("GCGTGCCTGGTCA$")

ACTGGCT$TGCGGC


### Reconstruct a String from its Burrows-Wheeler Transform

In [22]:
def index_symbol (column):
    idx_tuple = {symbol:0 for symbol in column}
    col = []
    for symbol in column:
        idx_tuple[symbol]+=1
        col.append((symbol, idx_tuple[symbol]))
    return col
def decodeFromBWT(bwt):
    first_col = ''.join(sorted([i for i in bwt]))
    first_index_tuple = index_symbol(first_col)
    last_col = bwt
    last_index_tuple = index_symbol(last_col)
    recons =''
    symbol = ('$',1)

    while len(recons)<len(bwt):
        symbol = first_index_tuple[last_index_tuple.index(symbol)]
        print("symbol -> ", symbol)
        recons += symbol[0]
    return recons[:-1]
    

In [24]:
decodeFromBWT("smnpbnnaaaaa$a")

symbol ->  ('p', 1)
symbol ->  ('a', 3)
symbol ->  ('n', 1)
symbol ->  ('a', 2)
symbol ->  ('m', 1)
symbol ->  ('a', 1)
symbol ->  ('b', 1)
symbol ->  ('a', 4)
symbol ->  ('n', 2)
symbol ->  ('a', 5)
symbol ->  ('n', 3)
symbol ->  ('a', 6)
symbol ->  ('s', 1)
symbol ->  ('$', 1)


'panamabananas'

### Generate the Last-to-First Mapping of a String

In [28]:
def lastTofirstMap(transform, idx):
    last_col = transform
    first_col = sorted(transform)
    last_index_tuple = index_symbol(last_col)
    first_index_tuple = index_symbol(first_col)

    mapped_idx = first_index_tuple.index(last_index_tuple[idx])
    return mapped_idx

lastTofirstMap("T$GACCA", 3)  

1

### Find All Occurrences of a Collection of Patterns in a String